#Layer Architecture Pattern

Data Access Layer

In [ ]:
import sqlite3

class DataAccessLayer:
    def __init__(self):
        self.connection = sqlite3.connect(':memory:')  # Create an in-memory SQLite database
        self.create_tables()

    def create_tables(self):
        # Create a Products table
        self.connection.execute('''
            CREATE TABLE IF NOT EXISTS Products (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                price REAL
            )
        ''')

    def add_product(self, product):
        # Insert a new product into the Products table
        cursor = self.connection.cursor()
        cursor.execute('''
            INSERT INTO Products (name, price) VALUES (?, ?)
        ''', (product['name'], product['price']))
        self.connection.commit()
        return cursor.lastrowid

    def get_product(self, product_id):
        # Retrieve a product by its ID
        cursor = self.connection.cursor()
        cursor.execute('''
            SELECT id, name, price FROM Products WHERE id = ?
        ''', (product_id,))
        return cursor.fetchone()

    def update_product(self, product_id, updated_product):
        # Update an existing product
        cursor = self.connection.cursor()
        cursor.execute('''
            UPDATE Products SET name = ?, price = ? WHERE id = ?
        ''', (updated_product['name'], updated_product['price'], product_id))
        self.connection.commit()

    def delete_product(self, product_id):
        # Delete a product by its ID
        cursor = self.connection.cursor()
        cursor.execute('''
            DELETE FROM Products WHERE id = ?
        ''', (product_id,))
        self.connection.commit()

    def get_all_products(self):
        # Retrieve all products
        cursor = self.connection.cursor()
        cursor.execute('''
            SELECT id, name, price FROM Products
        ''')
        return cursor.fetchall()

Business Logic Layer

In [ ]:
class BusinessLogicLayer:
    def __init__(self):
        self.data_access = DataAccessLayer()

    def process_data(self):
        print("=== Business Logic Layer ===")
        # Add a new product
        new_product_id = self.data_access.add_product({'name': 'Laptop', 'price': 1200.0})
        print(f"New product added with ID: {new_product_id}")

        # Retrieve a product
        product = self.data_access.get_product(new_product_id)
        print(f"Retrieved product: {product}")

        # Update the product
        updated_product = {'name': 'Laptop Pro', 'price': 1500.0}
        self.data_access.update_product(new_product_id, updated_product)
        print("Product updated successfully")

        # Retrieve all products
        all_products = self.data_access.get_all_products()
        print("All products:")
        for product in all_products:
            print(product)

        # Delete the product
        self.data_access.delete_product(new_product_id)
        print("Product deleted successfully")

Presentation Layer

In [ ]:
class PresentationLayer:
    def __init__(self):
        self.business_logic = BusinessLogicLayer()

    def run(self):
        print("=== Presentation Layer ===")
        choice = input("Enter command (e.g., 'process_data'): ")
        if choice == 'process_data':
            self.business_logic.process_data()
        else:
            print("Invalid command")

In [ ]:
# main.py
def main():
    presentation_layer = PresentationLayer()
    presentation_layer.run()

if __name__ == "__main__":
    main()

=== Presentation Layer ===
Enter command (e.g., 'process_data'): process_data
=== Business Logic Layer ===
New product added with ID: 1
Retrieved product: (1, 'Laptop', 1200.0)
Product updated successfully
All products:
(1, 'Laptop Pro', 1500.0)
Product deleted successfully


#Broker Pattern (Mediator Pattern)

Mediator (Broker) Implementation

In [ ]:
class ChatRoom:
    def __init__(self):
        self.users = []

    def register_user(self, user):
        self.users.append(user)

    def send_message(self, sender, message):
        for user in self.users:
            if user != sender:  # Do not send message back to the sender
                user.receive_message(sender, message)

    def broadcast_message(self, message):
        for user in self.users:
            user.receive_message("Broadcast", message)

Colleague (User) Implementation

In [ ]:
class User:
    def __init__(self, name, chat_room):
        self.name = name
        self.chat_room = chat_room
        self.chat_room.register_user(self)

    def send_message(self, message):
        self.chat_room.send_message(self, message)

    def broadcast_message(self, message):
        self.chat_room.broadcast_message(message)

    def receive_message(self, sender, message):
        print(f"{self.name} received message from {sender}: {message}")

In [ ]:
# Usage example
if __name__ == "__main__":
    # Create a chat room (mediator)
    chat_room = ChatRoom()

    # Create users (colleagues) and register them to the chat room (mediator)
    alice = User("Alice", chat_room)
    bob = User("Bob", chat_room)
    charlie = User("Charlie", chat_room)

    # User sends a message
    alice.send_message("Hello Bob!")  # Bob receives the message

    # User broadcasts a message
    charlie.broadcast_message("Hey everyone!")  # All users receive the broadcast message

Bob received message from <__main__.User object at 0x7ff5aaa00f40>: Hello Bob!
Charlie received message from <__main__.User object at 0x7ff5aaa00f40>: Hello Bob!
Alice received message from Broadcast: Hey everyone!
Bob received message from Broadcast: Hey everyone!
Charlie received message from Broadcast: Hey everyone!


#Proxy Pattern

Subject Interface (WeatherService)

In [ ]:
from abc import ABC, abstractmethod

class WeatherService(ABC):
    @abstractmethod
    def get_weather(self, city: str) -> str:
        pass

 Real Subject (WeatherProvider)

In [ ]:
class WeatherProvider(WeatherService):
    def get_weather(self, city: str) -> str:
        # Simulate fetching weather data from a remote API
        print(f"Fetching weather data for {city} from remote API...")
        # In a real implementation, this would make an actual API call
        return f"Weather in {city}: 25°C, Sunny"  # Simulated weather data

Proxy (WeatherProxy)

In [ ]:
class WeatherProxy(WeatherService):
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.weather_provider = WeatherProvider()
        self.cache = {}

    def authenticate(self):
        # Simulate authentication with API key
        print("Authenticating with API key...")
        # In a real implementation, this would perform actual authentication

    def get_weather(self, city: str) -> str:
        if city not in self.cache:
            self.authenticate()
            weather_data = self.weather_provider.get_weather(city)
            self.cache[city] = weather_data
        else:
            print(f"Fetching weather data for {city} from cache...")
            weather_data = self.cache[city]

        return weather_data

In [ ]:
if __name__ == "__main__":
    api_key = "your_api_key"
    weather_proxy = WeatherProxy(api_key)

    # Fetch weather data for cities (simulating multiple requests)
    print(weather_proxy.get_weather("New York"))
    print(weather_proxy.get_weather("London"))
    print(weather_proxy.get_weather("New York"))  # Fetch from cache
    print(weather_proxy.get_weather("Paris"))
    print(weather_proxy.get_weather("Paris"))  # Fetch from cache

Authenticating with API key...
Fetching weather data for New York from remote API...
Weather in New York: 25°C, Sunny
Authenticating with API key...
Fetching weather data for London from remote API...
Weather in London: 25°C, Sunny
Fetching weather data for New York from cache...
Weather in New York: 25°C, Sunny
Authenticating with API key...
Fetching weather data for Paris from remote API...
Weather in Paris: 25°C, Sunny
Fetching weather data for Paris from cache...
Weather in Paris: 25°C, Sunny


#Facade Pattern

In [ ]:
class AudioPlayer:
    def play_audio(self, audio_file):
        print(f"Playing audio: {audio_file}")

    def stop_audio(self):
        print("Stopping audio")

In [ ]:
class VideoPlayer:
    def play_video(self, video_file):
        print(f"Playing video: {video_file}")

    def stop_video(self):
        print("Stopping video")

In [ ]:
class SubtitleManager:
    def load_subtitle(self, subtitle_file):
        print(f"Loading subtitle: {subtitle_file}")

    def display_subtitle(self):
        print("Displaying subtitle")

    def hide_subtitle(self):
        print("Hiding subtitle")

In [ ]:
class MultimediaPlayerFacade:
    def __init__(self):
        self.audio_player = AudioPlayer()
        self.video_player = VideoPlayer()
        self.subtitle_manager = SubtitleManager()

    def play_movie(self, video_file, audio_file, subtitle_file=None):
        self.video_player.play_video(video_file)
        self.audio_player.play_audio(audio_file)
        if subtitle_file:
            self.subtitle_manager.load_subtitle(subtitle_file)
            self.subtitle_manager.display_subtitle()

    def stop_movie(self):
        self.video_player.stop_video()
        self.audio_player.stop_audio()
        self.subtitle_manager.hide_subtitle()

In [ ]:
if __name__ == "__main__":
    # Create multimedia player facade
    player = MultimediaPlayerFacade()

    # Play a movie
    print("Playing movie:")
    player.play_movie("movie.mp4", "audio.mp3", "subtitle.srt")

    # Simulate user interaction (e.g., pressing stop button)
    input("Press Enter to stop the movie...")

    # Stop the movie
    print("\nStopping movie...")
    player.stop_movie()

Playing movie:
Playing video: movie.mp4
Playing audio: audio.mp3
Loading subtitle: subtitle.srt
Displaying subtitle
Press Enter to stop the movie...

Stopping movie...
Stopping video
Stopping audio
Hiding subtitle


#Unit of Work Pattern

In [ ]:
class BankAccount:
    def __init__(self, account_id, balance):
        self.account_id = account_id
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount

    def withdraw(self, amount):
        if self.balance >= amount:
            self.balance -= amount
        else:
            raise ValueError("Insufficient balance")

    def __str__(self):
        return f"Account ID: {self.account_id}, Balance: ${self.balance}"

In [ ]:
class UnitOfWork:
    def __init__(self):
        self.new_objects = []
        self.dirty_objects = []
        self.removed_objects = []

    def register_new(self, obj):
        if obj not in self.new_objects:
            self.new_objects.append(obj)

    def register_dirty(self, obj):
        if obj not in self.dirty_objects and obj not in self.new_objects:
            self.dirty_objects.append(obj)

    def register_removed(self, obj):
        if obj not in self.removed_objects:
            self.removed_objects.append(obj)

    def commit(self):
        # Simulate committing changes to a database
        for obj in self.new_objects:
            print(f"Inserting new object: {obj}")
            # Perform actual database insert operation here

        for obj in self.dirty_objects:
            print(f"Updating object: {obj}")
            # Perform actual database update operation here

        for obj in self.removed_objects:
            print(f"Removing object: {obj}")
            # Perform actual database delete operation here

        # Clear the lists after commit
        self.new_objects.clear()
        self.dirty_objects.clear()
        self.removed_objects.clear()

In [ ]:
if __name__ == "__main__":
    # Create a unit of work instance
    unit_of_work = UnitOfWork()

    # Create new bank accounts
    account1 = BankAccount(account_id=1, balance=1000)
    account2 = BankAccount(account_id=2, balance=2000)

    # Register new bank accounts with the unit of work
    unit_of_work.register_new(account1)
    unit_of_work.register_new(account2)

    # Perform transactions
    try:
        account1.withdraw(500)
        account2.deposit(500)
        unit_of_work.register_dirty(account1)
        unit_of_work.register_dirty(account2)
    except ValueError as e:
        print(f"Transaction failed: {e}")

    # Commit changes
    unit_of_work.commit()

    # Display updated bank accounts
    print("Updated Bank Accounts:")
    print(account1)
    print(account2)

Inserting new object: Account ID: 1, Balance: $500
Inserting new object: Account ID: 2, Balance: $2500
Updated Bank Accounts:
Account ID: 1, Balance: $500
Account ID: 2, Balance: $2500


#Observer Pattern

Subject

In [ ]:
class NewsPublisher:
    def __init__(self):
        self._subscribers = []  # List to hold subscriber objects

    def register(self, subscriber):
        self._subscribers.append(subscriber)

    def unregister(self, subscriber):
        self._subscribers.remove(subscriber)

    def notify_subscribers(self, article):
        for subscriber in self._subscribers:
            subscriber.update(article)

Observer

In [ ]:
class Subscriber:
    def __init__(self, name):
        self.name = name

    def update(self, article):
        print(f"{self.name} received the latest news: {article}")

In [ ]:
if __name__ == "__main__":
    # Create a news publisher
    news_publisher = NewsPublisher()

    # Create subscribers
    subscriber1 = Subscriber("John")
    subscriber2 = Subscriber("Alice")

    # Register subscribers with the news publisher
    news_publisher.register(subscriber1)
    news_publisher.register(subscriber2)

    # Publish a new article
    article = "Breaking News: Python becomes the most popular programming language!"
    news_publisher.notify_subscribers(article)

    # Unregister a subscriber
    news_publisher.unregister(subscriber2)

    # Publish another article
    article2 = "New advancements in AI technology unveiled!"
    news_publisher.notify_subscribers(article2)

John received the latest news: Breaking News: Python becomes the most popular programming language!
Alice received the latest news: Breaking News: Python becomes the most popular programming language!
John received the latest news: New advancements in AI technology unveiled!


#ETL

Extract

In [ ]:
import csv

def extract_data_from_csv(csv_file):
    data = []
    with open(csv_file, 'r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
            data.append(row)
    return data

Transform

In [ ]:
def transform_data(data):
    transformed_data = []
    for row in data:
        # Example transformation: calculate total amount
        total_amount = float(row['Quantity']) * float(row['UnitPrice'])
        row['TotalAmount'] = total_amount
        transformed_data.append(row)
    return transformed_data

Load

In [ ]:
import sqlite3

def load_data_to_database(data, db_file):
    connection = sqlite3.connect(db_file)
    cursor = connection.cursor()

    # Create a table for the transformed data
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Orders (
            OrderID INTEGER PRIMARY KEY,
            CustomerID TEXT,
            Quantity INTEGER,
            UnitPrice REAL,
            TotalAmount REAL
        )
    ''')

    # Insert data into the database
    for row in data:
        cursor.execute('''
            INSERT INTO Orders (CustomerID, Quantity, UnitPrice, TotalAmount)
            VALUES (?, ?, ?, ?)
        ''', (row['CustomerID'], row['Quantity'], row['UnitPrice'], row['TotalAmount']))

    connection.commit()
    connection.close()

In [ ]:
%%writefile orders.csv
OrderID,CustomerID,Quantity,UnitPrice
1,C1001,3,10.5
2,C1002,5,20.0
3,C1003,2,15.75

Writing orders.csv


In [ ]:
if __name__ == "__main__":
    # Extract data from CSV
    csv_file = 'orders.csv'
    extracted_data = extract_data_from_csv(csv_file)

    # Transform the extracted data
    transformed_data = transform_data(extracted_data)

    # Load the transformed data into an SQLite database
    db_file = 'orders.db'
    load_data_to_database(transformed_data, db_file)

    print("ETL process completed successfully.")

ETL process completed successfully.


In [ ]:
import sqlite3

def select_orders_from_database(db_file):
    connection = sqlite3.connect(db_file)
    cursor = connection.cursor()

    # Execute SELECT query
    cursor.execute('''
        SELECT OrderID, CustomerID, Quantity, UnitPrice, TotalAmount
        FROM Orders
    ''')

    # Fetch and display results
    orders = cursor.fetchall()
    for order in orders:
        print(f"OrderID: {order[0]}, CustomerID: {order[1]}, Quantity: {order[2]}, "
              f"UnitPrice: {order[3]}, TotalAmount: {order[4]}")

    connection.close()

if __name__ == "__main__":
    # Specify the database file created during the ETL process
    db_file = 'orders.db'

    # Call the function to select and display orders from the database
    select_orders_from_database(db_file)

OrderID: 1, CustomerID: C1001, Quantity: 3, UnitPrice: 10.5, TotalAmount: 31.5
OrderID: 2, CustomerID: C1002, Quantity: 5, UnitPrice: 20.0, TotalAmount: 100.0
OrderID: 3, CustomerID: C1003, Quantity: 2, UnitPrice: 15.75, TotalAmount: 31.5


#Enterprise Service Bus

Producer

In [ ]:
import asyncio

# Message queue (simulated in-memory queue)
message_queue = []

async def produce_messages(num_messages):
    for i in range(1, num_messages + 1):
        message = f"Message {i}"
        message_queue.append(message)
        print(f"Produced: {message}")
        await asyncio.sleep(1)  # Simulate message production delay

# Run the producer coroutine
await produce_messages(5)  # Produce 5 messages

Produced: Message 1
Produced: Message 2
Produced: Message 3
Produced: Message 4
Produced: Message 5


Consumer

In [ ]:
async def consume_messages():
    while message_queue:
        message = message_queue.pop(0)  # Dequeue message
        print(f"Consumed: {message}")
        await asyncio.sleep(0.5)  # Simulate message processing delay

    print("All messages consumed. Consumer exiting...")

# Run the consumer coroutine
await consume_messages()

Consumed: Message 1
Consumed: Message 2
Consumed: Message 3
Consumed: Message 4
Consumed: Message 5
All messages consumed. Consumer exiting...
